<a href="https://colab.research.google.com/github/takky0330/ChatBot/blob/master/faiss_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#準備

In [1]:
## 以下の２つのバージョンを合わせる必要がある 2020/10/27
!pip3 install datascience==0.8.2
!pip3 install urllib3==1.25.4

## 初回の実行が終わったら、［ランタイム］ー［ランタイムの再起動］する

In [2]:
## MeCab のバージョンを合わせる
!pip3 install mecab-python3==0.996.5

     |████████████████████████████████| 17.1MB 243kB/s 


In [3]:
!git clone https://github.com/sonoisa/sentence-transformers
!cd sentence-transformers; pip install -r requirements.txt

Cloning into 'sentence-transformers'...
remote: Enumerating objects: 707, done.
remote: Total 707 (delta 0), reused 0 (delta 0), pack-reused 707
Receiving objects: 100% (707/707), 196.65 KiB | 694.00 KiB/s, done.
Resolving deltas: 100% (476/476), done.
     |████████████████████████████████| 450kB 7.9MB/s 
     |████████████████████████████████| 1.1MB 19.4MB/s 
     |████████████████████████████████| 890kB 37.4MB/s 
     |████████████████████████████████| 133kB 54.5MB/s 
     |████████████████████████████████| 6.7MB 53.3MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f0a9663539e357a7a6e7444d13bbceb1b72de022f7b190268d3f75b1c46ddfd1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
!wget -O sonobe-datasets-sentence-transformers-model.tar "https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=sonobe-datasets-sentence-transformers-model-2"
!tar -xvf sonobe-datasets-sentence-transformers-model.tar

--2020-10-28 05:46:08--  https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=sonobe-datasets-sentence-transformers-model-2
Resolving www.floydhub.com (www.floydhub.com)... 172.67.72.144, 104.26.0.30, 104.26.1.30, ...
Connecting to www.floydhub.com (www.floydhub.com)|172.67.72.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/tar]
Saving to: ‘sonobe-datasets-sentence-transformers-model.tar’

sonobe-datasets-sen     [            <=>     ] 422.28M  21.1MB/s    in 20s     

2020-10-28 05:46:28 (21.7 MB/s) - ‘sonobe-datasets-sentence-transformers-model.tar’ saved [442788352]

./
./training_bert_japanese/
./training_bert_japanese/0_BERTJapanese/
./training_bert_japanese/0_BERTJapanese/added_tokens.json
./training_bert_japanese/0_BERTJapanese/config.json
./training_bert_japanese/0_BERTJapanese/pytorch_model.bin
./training_bert_japanese/0_BERTJapanese/sentence_bert_config.json
./training_bert_ja

In [5]:
%cd sentence-transformers

/content/sentence-transformers


In [6]:
%tensorflow_version 2.x
from sentence_transformers import SentenceTransformer
import numpy as np

model_path = "/content/training_bert_japanese"
model = SentenceTransformer(model_path, show_progress_bar=False)

#検索対象の embed

In [7]:
#sentences = ["お辞儀をしている男性会社員", "笑い袋", "テクニカルエバンジェリスト（女性）", "戦うAI", "笑う男性（5段階）", "漫才師", "お辞儀をしている医者（女性）", "お辞儀をしている薬剤師", "福笑いをしている人", "AIの家族", "コント師", "福笑い（女性）", "お辞儀をしている犬", "苦笑いをする女性", "お辞儀をしている医者", "いろいろな漫符", 
#"雛人形「仕丁・三人上戸」", "ダンス「踊る男性」", "拍手をしている人", "定年（男性）", "ものまね芸人", "福笑いのおたふく","お辞儀をしている看護師（男性）", "愛想笑い", "福笑い（ひょっとこ）", "成長する人工知能", "苦笑いをする男性", "運動会「徒競走・白組」", "人工知能と喧嘩をする人", "人工知能", "ありがた迷惑", "お辞儀をしているクマ", "笑う女性（5段階）", 
#"人工知能とメールをする人（男性）", "技術書", "笑いをこらえる人（女性）", "ダンス「踊る女性」", "お辞儀をしている猫", "福笑い（男性）", "武器を持つAI", "作曲する人工知能", "縄跳びを飛んでいる女性", "福笑い（おかめ）", "茅の輪くぐり", "表情", "AIと仲良くなる人間", "お笑い芸人「漫才師」", "人工知能とメールをする人（女性）", "人工知能と戦う囲碁の棋士", "拍手している女の子", 
#"検索する人工知能", "ピースサインを出す人（女性）", "啓示を受けた人（女性）", "仕事をする人工知能", "一輪車に乗る女の子", "お辞儀をしているウサギ", "走る猫（笑顔）", "人工知能と戦う将棋の棋士", "遠足「お弁当・男の子・女の子」", "心を持ったAI", "プレゼントをもらって喜ぶ女の子", "技術書（バラバラ）", "いろいろな表情の酔っぱらい（男性）", "拍手している人（棒人間）", 
#"仕事を奪う人工知能", "文章を書く人工知能", "いろいろな映画の「つづく」", "絵を描く人工知能", "拍手している男の子", "ハリセン", "人工知能と仲良くする人たち", "ON AIRランプ", "いろいろな表情の酔っぱらい（女性）", "徹夜明けの笑顔（女性）", "徹夜明けの笑顔（男性）", "お辞儀をしている女性会社員", "バンザイをしているお婆さん", "画像認識をするAI", 
#"芸人の男の子（将来の夢）", "料理「女性」", "ピコピコハンマー", "鏡を見る人（笑顔の男性）", "笑いをこらえる人（男性）", "シンギュラリティ", "人工知能に仕事を任せる人", "スマートスピーカー", "学ぶ人工知能", "人工知能・AI", "英語のアルファベット", "お金を見つめてニヤけている男性", "「ありがとう」と言っている人", "定年（女性）", "テクニカルエバンジェリスト（男性）", 
#"スタンディングオベーション"]

#queries = ["武器を持つAI", '暴走したAI', '暴走した人工知能', 'いらすとやさんに感謝', 'つづく']

In [8]:
import pandas as pd
sentences_csv = pd.read_csv("https://drive.google.com/uc?id=1h7Nj-X8WY9OQnaRunAr02Xl6S74eXzKC", sep=",")
sentences = list(sentences_csv['EXP'])
queries_csv = pd.read_csv("https://drive.google.com/uc?id=1i-d3KxmRVoGnAkvXhrLu9mrCtv_9_qLY", sep=",")
queries = list(queries_csv["Query"])

In [23]:
sentence_vectors = model.encode(sentences)
print(len(sentence_vectors))
print([len(v) for v in sentence_vectors])
query_embeddings = model.encode(queries)
print(len(queries_csv))
print([len(v) for v in query_embeddings])

72
[768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768]
15
[768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768, 768]


#検証

## cos類似度

In [84]:
import scipy.spatial
import time

closest_n = [1]

start = time.time()
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_vectors, metric="cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n======================")
    print("Query:", query)

    for i, (idx, distance) in enumerate(results[0:max(closest_n) + 1 ]):
        if i + 1 in closest_n:
            print(i + 1, sentences[idx].strip(), "(Score: %.4f)" % (1 - distance))

elapsed_time = time.time() - start
print("\n\n -------------------------------------------------------------\n elapsed_time:{0}".format(elapsed_time) + "[sec]")


Query: RobotDiverの操作方法について教えてください
1 RobotDiverの利用方法について教えて下さい (Score: 0.8767)

Query: どのように利用するのでしょう？
1 RobotDiverはどのように利用するんですか？ (Score: 0.6096)

Query: 何ができるのですか？
1 Machine Leaningを利用すると予測ができるのでしょうか (Score: 0.6004)

Query: マニュアルはありますか？
1 利用するマニュアルは有りますか (Score: 0.9086)

Query: TangoACとはどんなことができるのですか？
1 TangoACについて知りたい (Score: 0.7564)

Query: 動きません
1 TangoACの結果が出ません (Score: 0.6120)

Query: 動かん
1 ウインドウズでも動かせますか (Score: 0.5284)

Query: URLは？
1 ロボットダイバーのURLを教えて下さい (Score: 0.4726)

Query: 問い合わせは可能ですか？
1 電話での問い合わせは可能ですか (Score: 0.7921)

Query: メールでの問い合わせは出来ますか？
1 メールでの問い合わせは可能ですか (Score: 0.9732)

Query: アフターコーディングの機能はありますか？
1 Machine Leaningを利用すると予測ができるのでしょうか (Score: 0.5088)

Query: 動作環境
1 RobotDiverの動作環境は何処かに記載されていますか (Score: 0.6359)

Query: 推奨のブラウザは何？
1 どのようなブラウザが利用できますか (Score: 0.8274)

Query: Macでも利用できるんの？
1 Macで動かすことは可能ですか (Score: 0.9235)

Query: インストールは必要？
1 ロボットダイバーはインストールが必要ですか (Score: 0.5900)


 -------------------------------------------------------------
 elapsed_time:0.0312418

## faiss CPU

In [85]:
!pip3 install faiss-cpu

In [86]:
import time
import numpy as np
import faiss

### IndexFlatL2

In [87]:
d = max([len(v) for v in sentence_vectors])
index = faiss.IndexFlatL2(d)
index.add(np.array(sentence_vectors).astype('float32'))

In [88]:
closest_n = [1]
start = time.time()

D, I = index.search(np.array(query_embeddings).astype('float32'), max(closest_n))
for _I, (IDX, DISTANCE) in enumerate(zip(I, D)):
    print("\n======================")
    print("Query:", queries[_I])

    for i, (idx, distance) in enumerate(zip(IDX, DISTANCE)):
        if i + 1 in closest_n:
            print(i + 1, sentences[idx].strip(), "(Score: %.4f)" % (1 - distance))

elapsed_time = time.time() - start
print("\n\n -------------------------------------------------------------\n elapsed_time:{0}".format(elapsed_time) + "[sec]")


Query: RobotDiverの操作方法について教えてください
1 RobotDiverの利用方法について教えて下さい (Score: -89.7067)

Query: どのように利用するのでしょう？
1 RobotDiverはどのように利用するんですか？ (Score: -294.2201)

Query: 何ができるのですか？
1 Machine Leaningを利用すると予測ができるのでしょうか (Score: -300.3735)

Query: マニュアルはありますか？
1 利用するマニュアルは有りますか (Score: -81.9835)

Query: TangoACとはどんなことができるのですか？
1 TangoACについて知りたい (Score: -186.4214)

Query: 動きません
1 TangoACの結果が出ません (Score: -316.1909)

Query: 動かん
1 ウインドウズでも動かせますか (Score: -400.8091)

Query: URLは？
1 ロボットダイバーのURLを教えて下さい (Score: -418.8790)

Query: 問い合わせは可能ですか？
1 電話での問い合わせは可能ですか (Score: -170.3867)

Query: メールでの問い合わせは出来ますか？
1 メールでの問い合わせは可能ですか (Score: -21.1982)

Query: アフターコーディングの機能はありますか？
1 Machine Leaningを利用すると予測ができるのでしょうか (Score: -348.0704)

Query: 動作環境
1 RobotDiverの動作環境は何処かに記載されていますか (Score: -307.5202)

Query: 推奨のブラウザは何？
1 どのようなブラウザが利用できますか (Score: -142.9253)

Query: Macでも利用できるんの？
1 Macで動かすことは可能ですか (Score: -61.8551)

Query: インストールは必要？
1 ロボットダイバーはインストールが必要ですか (Score: -329.1868)


 --------------------------------------------

### IndexFlatIP

In [89]:
d = max([len(v) for v in sentence_vectors])
index = faiss.IndexFlatIP(d)
index.add(np.array(sentence_vectors).astype('float32'))

In [90]:
closest_n = [1]
start = time.time()

D, I = index.search(np.array(query_embeddings).astype('float32'), max(closest_n))
for _I, (IDX, DISTANCE) in enumerate(zip(I, D)):
    print("\n======================")
    print("Query:", queries[_I])

    for i, (idx, distance) in enumerate(zip(IDX, DISTANCE)):
        if i + 1 in closest_n:
            print(i + 1, sentences[idx].strip(), "(Score: %.4f)" % (1 - distance))

elapsed_time = time.time() - start
print("\n\n -------------------------------------------------------------\n elapsed_time:{0}".format(elapsed_time) + "[sec]")


Query: RobotDiverの操作方法について教えてください
1 RobotDiverの利用方法について教えて下さい (Score: -321.2726)

Query: どのように利用するのでしょう？
1 アカウントはどのように取得するんですか (Score: -239.6324)

Query: 何ができるのですか？
1 Machine Leaningを利用すると予測ができるのでしょうか (Score: -219.0443)

Query: マニュアルはありますか？
1 利用するマニュアルは有りますか (Score: -411.4138)

Query: TangoACとはどんなことができるのですか？
1 TangoACについて知りたい (Score: -286.8451)

Query: 動きません
1 TangoACの結果が出ません (Score: -246.5840)

Query: 動かん
1 ウインドウズでも動かせますか (Score: -223.8102)

Query: URLは？
1 要約とはなんですか (Score: -186.9919)

Query: 問い合わせは可能ですか？
1 電話での問い合わせは可能ですか (Score: -324.4099)

Query: メールでの問い合わせは出来ますか？
1 メールでの問い合わせは可能ですか (Score: -398.6963)

Query: アフターコーディングの機能はありますか？
1 ウインドウズでも動かせますか (Score: -197.3774)

Query: 動作環境
1 RobotDiverの動作環境は何処かに記載されていますか (Score: -263.1935)

Query: 推奨のブラウザは何？
1 どのようなブラウザが利用できますか (Score: -343.8755)

Query: Macでも利用できるんの？
1 Macで動かすことは可能ですか (Score: -377.5128)

Query: インストールは必要？
1 ロボットダイバーはインストールが必要ですか (Score: -235.1591)


 -------------------------------------------------------------
 elapsed_time

### IndexIVFFlat

In [105]:
d = max([len(v) for v in sentence_vectors])
nlist = 72   ## いろんな事例では 100 にしている事例が多いが、100 だとエラー
k = 4
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
index.train(np.array(sentence_vectors).astype('float32'))
index.add(np.array(sentence_vectors).astype('float32'))

In [106]:
closest_n = [1]
start = time.time()

#index.nprobe = 1
D, I = index.search(np.array(query_embeddings).astype('float32'), max(closest_n))
for _I, (IDX, DISTANCE) in enumerate(zip(I, D)):
    print("\n======================")
    print("Query:", queries[_I])

    for i, (idx, distance) in enumerate(zip(IDX, DISTANCE)):
        if i + 1 in closest_n:
            print(i + 1, sentences[idx].strip(), "(Score: %.4f)" % (1 - distance))

elapsed_time = time.time() - start
print("\n\n -------------------------------------------------------------\n elapsed_time:{0}".format(elapsed_time) + "[sec]")


Query: RobotDiverの操作方法について教えてください
1 RobotDiverの利用方法について教えて下さい (Score: -89.7067)

Query: どのように利用するのでしょう？
1 RobotDiverはどのように利用するんですか？ (Score: -294.2201)

Query: 何ができるのですか？
1 Machine Leaningを利用すると予測ができるのでしょうか (Score: -300.3735)

Query: マニュアルはありますか？
1 利用するマニュアルは有りますか (Score: -81.9835)

Query: TangoACとはどんなことができるのですか？
1 TangoACについて知りたい (Score: -186.4214)

Query: 動きません
1 TangoACの結果が出ません (Score: -316.1909)

Query: 動かん
1 ウインドウズでも動かせますか (Score: -400.8091)

Query: URLは？
1 ロボットダイバーのURLを教えて下さい (Score: -418.8790)

Query: 問い合わせは可能ですか？
1 電話での問い合わせは可能ですか (Score: -170.3867)

Query: メールでの問い合わせは出来ますか？
1 メールでの問い合わせは可能ですか (Score: -21.1982)

Query: アフターコーディングの機能はありますか？
1 Machine Leaningを利用すると予測ができるのでしょうか (Score: -348.0704)

Query: 動作環境
1 RobotDiverの動作環境は何処かに記載されていますか (Score: -307.5202)

Query: 推奨のブラウザは何？
1 どのようなブラウザが利用できますか (Score: -142.9253)

Query: Macでも利用できるんの？
1 Macで動かすことは可能ですか (Score: -61.8551)

Query: インストールは必要？
1 ロボットダイバーはインストールが必要ですか (Score: -329.1868)


 --------------------------------------------

### IndexIVFFlat（GPU）

In [109]:
#GPUは使わない想定なので、今回は割愛！